In [1]:
import pandas as pd

## Pandas Dataframes

In previous years we ran this course using R, however this year we will continue to work with Python. As you have learned, the most common library for manipulating structured data in Python is called `pandas`. This notebook covers a quick refresher and tour, using some of the data we will work with later in this course.

There are a lot of pandas resources available, eg [cheatsheets](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf).

The data that we have just loaded comes from the file containing the Arthurian manuscripts metadata. Each row represents a single manuscript and the columns describe different properties of each of these books (or at least, what remains of them).

First, let's load the data from a CSV file on disk.

In [2]:
df = pd.read_csv("../../datasets/arthur/manuscripts.csv", index_col=0)
df

,signature,repository,script,text-height,text-width,text-src,illustrations,date_min,date_max,leaf-height,leaf-width,leaf-src,number-lines,material,physical-type,country,region,columns
1,"Schloss Anholt (Isselburg), Fürstl. Salm-Salm'...","Schloss Anholt (Isselburg), Fürstl. Salm-Salm'...",textualis,220.0,164.0,original,0,1320.0,1325.0,308.0,215.0,current,32.0,parchment,fragment,NaN,NaN,2.0
2,"Berlin, Staatsbibliothek, germ. fol. 923, 38","Berlin, Staatsbibliothek",NaN,NaN,NaN,NaN,0,1300.0,1400.0,NaN,NaN,NaN,NaN,parchment,fragment,Germany,Bayern,NaN
3,"Linz, Landesarchiv, 96/IV/54","Linz, Landesarchiv",NaN,212.0,163.0,original,0,1390.0,1410.0,280.0,200.0,current,31.0,paper,codex,NaN,NaN,2.0
4,"Brussels, Koninklijke Bibliotheek, II 115,2","Brussels, Koninklijke Bibliotheek",textualis,185.0,132.0,original,0,1340.0,1360.0,230.0,155.0,original,39.0,parchment,fragment,Southern Netherlands,Flanders,2.0
5,"Munich, Staatsbibliothek, cgm. 5249 / 3b","Munich, Staatsbibliothek",textualis,195.0,127.0,original,0,1275.0,1300.0,232.0,160.0,current,42.0,parchment,fragment,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162,"Orléans, Bibliothèque Municipale, -","Orléans, Bibliothèque Municipale",NaN,NaN,NaN,NaN,0,1200.0,1300.0,150.0,105.0,current,30.0,parchment,fragment,NaN,NaN,1.0
1163,"Freiburg, Universitätsbibliothek, 513","Freiburg, Universitätsbibliothek",NaN,150.0,115.0,original,0,1290.0,1310.0,160.0,140.0,current,NaN,parchment,fragment,NaN,NaN,2.0
1164,"Paris, Bibliothèque Nationale de France, fr. 340","Paris, Bibliothèque Nationale de France",NaN,NaN,NaN,NaN,76,1400.0,1425.0,420.0,300.0,current,53.0,parchment,codex,NaN,NaN,3.0
1165,"Heidelberg, Universitätsbibliothek, Cpg 397","Heidelberg, Universitätsbibliothek",textualis,165.0,100.0,original,0,1225.0,1250.0,198.0,122.0,current,26.0,parchment,codex,Germany,Rheinfranken,1.0


We can use the `shape` property to find the dimensions of the `DataFrame` object (rows x columns). Remember that Python *properties* are accessed as bare words (`df.shape` not `df.shape()`), whereas *methods* must have parentheses, even when no arguments are given.

In [3]:
df.shape

(1166, 18)

There are many other useful properties. We can look at the `columns`, and later we will make a lot of use of the `index`

In [4]:
df.columns

Index(['signature', 'repository', 'script', 'text-height', 'text-width',
       'text-src', 'illustrations', 'date_min', 'date_max', 'leaf-height',
       'leaf-width', 'leaf-src', 'number-lines', 'material', 'physical-type',
       'country', 'region', 'columns'],
      dtype='object')

### Basic Methods

We can take a quick look at the DataFrame using the `head()` and `tail()` methods, but also note how Jupyter notebooks will already return a basic view of the DataFrame as long as it is the last thing evaluated in a cell, so mostly we will use that 'shortcut' below.

In [5]:
df.head(3)

,signature,repository,script,text-height,text-width,text-src,illustrations,date_min,date_max,leaf-height,leaf-width,leaf-src,number-lines,material,physical-type,country,region,columns
1,"Schloss Anholt (Isselburg), Fürstl. Salm-Salm'...","Schloss Anholt (Isselburg), Fürstl. Salm-Salm'...",textualis,220.0,164.0,original,0,1320.0,1325.0,308.0,215.0,current,32.0,parchment,fragment,NaN,NaN,2.0
2,"Berlin, Staatsbibliothek, germ. fol. 923, 38","Berlin, Staatsbibliothek",NaN,NaN,NaN,NaN,0,1300.0,1400.0,NaN,NaN,NaN,NaN,parchment,fragment,Germany,Bayern,NaN
3,"Linz, Landesarchiv, 96/IV/54","Linz, Landesarchiv",NaN,212.0,163.0,original,0,1390.0,1410.0,280.0,200.0,current,31.0,paper,codex,NaN,NaN,2.0


In [6]:
df.tail(2)

,signature,repository,script,text-height,text-width,text-src,illustrations,date_min,date_max,leaf-height,leaf-width,leaf-src,number-lines,material,physical-type,country,region,columns
1165,"Heidelberg, Universitätsbibliothek, Cpg 397","Heidelberg, Universitätsbibliothek",textualis,165.0,100.0,original,0,1225.0,1250.0,198.0,122.0,current,26.0,parchment,codex,Germany,Rheinfranken,1.0
1166,"Florence, Biblioteca Laurenziana, Plut. 77, 23","Florence, Biblioteca Laurenziana",NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,paper,codex,Italy,NaN,NaN


In [7]:
# the last object evaluated in a cell is outputted by Jupyter

df

,signature,repository,script,text-height,text-width,text-src,illustrations,date_min,date_max,leaf-height,leaf-width,leaf-src,number-lines,material,physical-type,country,region,columns
1,"Schloss Anholt (Isselburg), Fürstl. Salm-Salm'...","Schloss Anholt (Isselburg), Fürstl. Salm-Salm'...",textualis,220.0,164.0,original,0,1320.0,1325.0,308.0,215.0,current,32.0,parchment,fragment,NaN,NaN,2.0
2,"Berlin, Staatsbibliothek, germ. fol. 923, 38","Berlin, Staatsbibliothek",NaN,NaN,NaN,NaN,0,1300.0,1400.0,NaN,NaN,NaN,NaN,parchment,fragment,Germany,Bayern,NaN
3,"Linz, Landesarchiv, 96/IV/54","Linz, Landesarchiv",NaN,212.0,163.0,original,0,1390.0,1410.0,280.0,200.0,current,31.0,paper,codex,NaN,NaN,2.0
4,"Brussels, Koninklijke Bibliotheek, II 115,2","Brussels, Koninklijke Bibliotheek",textualis,185.0,132.0,original,0,1340.0,1360.0,230.0,155.0,original,39.0,parchment,fragment,Southern Netherlands,Flanders,2.0
5,"Munich, Staatsbibliothek, cgm. 5249 / 3b","Munich, Staatsbibliothek",textualis,195.0,127.0,original,0,1275.0,1300.0,232.0,160.0,current,42.0,parchment,fragment,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162,"Orléans, Bibliothèque Municipale, -","Orléans, Bibliothèque Municipale",NaN,NaN,NaN,NaN,0,1200.0,1300.0,150.0,105.0,current,30.0,parchment,fragment,NaN,NaN,1.0
1163,"Freiburg, Universitätsbibliothek, 513","Freiburg, Universitätsbibliothek",NaN,150.0,115.0,original,0,1290.0,1310.0,160.0,140.0,current,NaN,parchment,fragment,NaN,NaN,2.0
1164,"Paris, Bibliothèque Nationale de France, fr. 340","Paris, Bibliothèque Nationale de France",NaN,NaN,NaN,NaN,76,1400.0,1425.0,420.0,300.0,current,53.0,parchment,codex,NaN,NaN,3.0
1165,"Heidelberg, Universitätsbibliothek, Cpg 397","Heidelberg, Universitätsbibliothek",textualis,165.0,100.0,original,0,1225.0,1250.0,198.0,122.0,current,26.0,parchment,codex,Germany,Rheinfranken,1.0


If we access an individual column we will get a `Series`. This can also be done with a magic method that treats the column name like a property. The two calls below are mostly equivalent (sometimes the longer string subscript access is required)

In [8]:
df.script

1       textualis
2             NaN
3             NaN
4       textualis
5       textualis
          ...    
1162          NaN
1163          NaN
1164          NaN
1165    textualis
1166          NaN
Name: script, Length: 1166, dtype: object

In [9]:
df["script"]

1       textualis
2             NaN
3             NaN
4       textualis
5       textualis
          ...    
1162          NaN
1163          NaN
1164          NaN
1165    textualis
1166          NaN
Name: script, Length: 1166, dtype: object

Most methods on a DataFrame will return a new DataFrame. Here we pull out the width and height of the pages (leaves) for those manuscripts where the data is recorded. Since that method returns a `DataFrame` we can immediately *chain* another method, `dropna()`, to drop all rows where either value is NA.

NOTE CAREFULLY: The indices for the rows *do not change* (unless we force it to by using a different method). That means our indices are no longer sequential. This is an advantage!

In [10]:
dimensions = df[["leaf-height", "leaf-width"]].dropna()
dimensions.index

Index([   1,    3,    4,    5,    9,   12,   15,   16,   17,   21,
       ...
       1154, 1155, 1157, 1158, 1159, 1160, 1162, 1163, 1164, 1165],
      dtype='int64', length=727)

In [11]:
idx = dimensions.index

In [12]:
type(idx)

pandas.core.indexes.base.Index

In [13]:
df[["leaf-height", "leaf-width"]]

,leaf-height,leaf-width
1,308.0,215.0
2,NaN,NaN
3,280.0,200.0
4,230.0,155.0
5,232.0,160.0
...,...,...
1162,150.0,105.0
1163,160.0,140.0
1164,420.0,300.0
1165,198.0,122.0


Here we can pull the rows from the original dataframe, but ONLY at the index locations (`.loc`) specified in the `dimensions` dataframe. Note how we now have a dataframe with only 727 entries.

In [14]:
df.loc[idx]

,signature,repository,script,text-height,text-width,text-src,illustrations,date_min,date_max,leaf-height,leaf-width,leaf-src,number-lines,material,physical-type,country,region,columns
1,"Schloss Anholt (Isselburg), Fürstl. Salm-Salm'...","Schloss Anholt (Isselburg), Fürstl. Salm-Salm'...",textualis,220.0,164.0,original,0,1320.0,1325.0,308.0,215.0,current,32.0,parchment,fragment,NaN,NaN,2.0
3,"Linz, Landesarchiv, 96/IV/54","Linz, Landesarchiv",NaN,212.0,163.0,original,0,1390.0,1410.0,280.0,200.0,current,31.0,paper,codex,NaN,NaN,2.0
4,"Brussels, Koninklijke Bibliotheek, II 115,2","Brussels, Koninklijke Bibliotheek",textualis,185.0,132.0,original,0,1340.0,1360.0,230.0,155.0,original,39.0,parchment,fragment,Southern Netherlands,Flanders,2.0
5,"Munich, Staatsbibliothek, cgm. 5249 / 3b","Munich, Staatsbibliothek",textualis,195.0,127.0,original,0,1275.0,1300.0,232.0,160.0,current,42.0,parchment,fragment,NaN,NaN,2.0
9,"Escorial, Real Biblioteca del Monasterio, M. I...","Escorial, Real Biblioteca del Monasterio",textualis,224.0,128.0,original,0,1275.0,1300.0,260.0,128.0,current,45.0,parchment,codex,France,Normandy,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160,"Marburg, Staatsarchiv, Best. 147 Hr 1, 16","Marburg, Staatsarchiv",NaN,250.0,190.0,original,0,1200.0,1300.0,95.0,160.0,original,43.0,parchment,fragment,NaN,NaN,2.0
1162,"Orléans, Bibliothèque Municipale, -","Orléans, Bibliothèque Municipale",NaN,NaN,NaN,NaN,0,1200.0,1300.0,150.0,105.0,current,30.0,parchment,fragment,NaN,NaN,1.0
1163,"Freiburg, Universitätsbibliothek, 513","Freiburg, Universitätsbibliothek",NaN,150.0,115.0,original,0,1290.0,1310.0,160.0,140.0,current,NaN,parchment,fragment,NaN,NaN,2.0
1164,"Paris, Bibliothèque Nationale de France, fr. 340","Paris, Bibliothèque Nationale de France",NaN,NaN,NaN,NaN,76,1400.0,1425.0,420.0,300.0,current,53.0,parchment,codex,NaN,NaN,3.0


### Operating on Columns

We can insert new `Series` objects (or actually even normal lists, or numpy arrays, etc) into the dataframe as columns. Here, we create one column using a vectorized boolean comparison, and another one which we create by multiplying two existing columns. Once again note how these are *vector* operations -- `ColA * ColB` multiplies the matching values in every row, all in one statement.

Note how in the automatic view the reported shape is increasing by one column each time....

In [15]:
(df.illustrations > 0)

1       False
2       False
3       False
4       False
5       False
        ...  
1162    False
1163    False
1164     True
1165    False
1166    False
Name: illustrations, Length: 1166, dtype: bool

In [16]:
# insert takes index-to-insert, column-name, Series as positional arguments

df.insert(3, "illustrated", df.illustrations > 0)
df

,signature,repository,script,illustrated,text-height,text-width,text-src,illustrations,date_min,date_max,leaf-height,leaf-width,leaf-src,number-lines,material,physical-type,country,region,columns
1,"Schloss Anholt (Isselburg), Fürstl. Salm-Salm'...","Schloss Anholt (Isselburg), Fürstl. Salm-Salm'...",textualis,False,220.0,164.0,original,0,1320.0,1325.0,308.0,215.0,current,32.0,parchment,fragment,NaN,NaN,2.0
2,"Berlin, Staatsbibliothek, germ. fol. 923, 38","Berlin, Staatsbibliothek",NaN,False,NaN,NaN,NaN,0,1300.0,1400.0,NaN,NaN,NaN,NaN,parchment,fragment,Germany,Bayern,NaN
3,"Linz, Landesarchiv, 96/IV/54","Linz, Landesarchiv",NaN,False,212.0,163.0,original,0,1390.0,1410.0,280.0,200.0,current,31.0,paper,codex,NaN,NaN,2.0
4,"Brussels, Koninklijke Bibliotheek, II 115,2","Brussels, Koninklijke Bibliotheek",textualis,False,185.0,132.0,original,0,1340.0,1360.0,230.0,155.0,original,39.0,parchment,fragment,Southern Netherlands,Flanders,2.0
5,"Munich, Staatsbibliothek, cgm. 5249 / 3b","Munich, Staatsbibliothek",textualis,False,195.0,127.0,original,0,1275.0,1300.0,232.0,160.0,current,42.0,parchment,fragment,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162,"Orléans, Bibliothèque Municipale, -","Orléans, Bibliothèque Municipale",NaN,False,NaN,NaN,NaN,0,1200.0,1300.0,150.0,105.0,current,30.0,parchment,fragment,NaN,NaN,1.0
1163,"Freiburg, Universitätsbibliothek, 513","Freiburg, Universitätsbibliothek",NaN,False,150.0,115.0,original,0,1290.0,1310.0,160.0,140.0,current,NaN,parchment,fragment,NaN,NaN,2.0
1164,"Paris, Bibliothèque Nationale de France, fr. 340","Paris, Bibliothèque Nationale de France",NaN,True,NaN,NaN,NaN,76,1400.0,1425.0,420.0,300.0,current,53.0,parchment,codex,NaN,NaN,3.0
1165,"Heidelberg, Universitätsbibliothek, Cpg 397","Heidelberg, Universitätsbibliothek",textualis,False,165.0,100.0,original,0,1225.0,1250.0,198.0,122.0,current,26.0,parchment,codex,Germany,Rheinfranken,1.0


In [17]:
df.insert(4, "surface", df["leaf-height"] * df["leaf-width"])
df

,signature,repository,script,illustrated,surface,text-height,text-width,text-src,illustrations,date_min,date_max,leaf-height,leaf-width,leaf-src,number-lines,material,physical-type,country,region,columns
1,"Schloss Anholt (Isselburg), Fürstl. Salm-Salm'...","Schloss Anholt (Isselburg), Fürstl. Salm-Salm'...",textualis,False,66220.0,220.0,164.0,original,0,1320.0,1325.0,308.0,215.0,current,32.0,parchment,fragment,NaN,NaN,2.0
2,"Berlin, Staatsbibliothek, germ. fol. 923, 38","Berlin, Staatsbibliothek",NaN,False,NaN,NaN,NaN,NaN,0,1300.0,1400.0,NaN,NaN,NaN,NaN,parchment,fragment,Germany,Bayern,NaN
3,"Linz, Landesarchiv, 96/IV/54","Linz, Landesarchiv",NaN,False,56000.0,212.0,163.0,original,0,1390.0,1410.0,280.0,200.0,current,31.0,paper,codex,NaN,NaN,2.0
4,"Brussels, Koninklijke Bibliotheek, II 115,2","Brussels, Koninklijke Bibliotheek",textualis,False,35650.0,185.0,132.0,original,0,1340.0,1360.0,230.0,155.0,original,39.0,parchment,fragment,Southern Netherlands,Flanders,2.0
5,"Munich, Staatsbibliothek, cgm. 5249 / 3b","Munich, Staatsbibliothek",textualis,False,37120.0,195.0,127.0,original,0,1275.0,1300.0,232.0,160.0,current,42.0,parchment,fragment,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162,"Orléans, Bibliothèque Municipale, -","Orléans, Bibliothèque Municipale",NaN,False,15750.0,NaN,NaN,NaN,0,1200.0,1300.0,150.0,105.0,current,30.0,parchment,fragment,NaN,NaN,1.0
1163,"Freiburg, Universitätsbibliothek, 513","Freiburg, Universitätsbibliothek",NaN,False,22400.0,150.0,115.0,original,0,1290.0,1310.0,160.0,140.0,current,NaN,parchment,fragment,NaN,NaN,2.0
1164,"Paris, Bibliothèque Nationale de France, fr. 340","Paris, Bibliothèque Nationale de France",NaN,True,126000.0,NaN,NaN,NaN,76,1400.0,1425.0,420.0,300.0,current,53.0,parchment,codex,NaN,NaN,3.0
1165,"Heidelberg, Universitätsbibliothek, Cpg 397","Heidelberg, Universitätsbibliothek",textualis,False,24156.0,165.0,100.0,original,0,1225.0,1250.0,198.0,122.0,current,26.0,parchment,codex,Germany,Rheinfranken,1.0


### Just Checking...

Let's just see if the first entry by *position* (not index, `iloc` always uses sequential position) has been correctly calculated. This is not exhaustive, but it is a good habit to make sure that things look like they are doing what they should be doing.

In [18]:
dimensions.iloc[0]

leaf-height    308.0
leaf-width     215.0
Name: 1, dtype: float64

In [19]:
308 * 215 == df.iloc[0]["surface"]

True

### More index tricks

Now we'll save the new dataframe we get by accessing the original `df` at the indices from the `dimensions` df, into a new variable called `df_valid`

In [20]:
df_valid = df.loc[dimensions.index]

Sorting is easy in pandas, using `sort_values` (this sorts the whole row, using the values in the column you provide)

In [21]:
df_valid.sort_values(by="surface")

,signature,repository,script,illustrated,surface,text-height,text-width,text-src,illustrations,date_min,date_max,leaf-height,leaf-width,leaf-src,number-lines,material,physical-type,country,region,columns
501,"Brussels, Koninklijke Bibliotheek, 9086-9087 [...","Brussels, Koninklijke Bibliotheek",NaN,False,0.0,NaN,NaN,NaN,0,1400.0,1500.0,398.0,0.0,current,NaN,paper,codex,NaN,NaN,NaN
988,"Oxford, Bodleian Library, Michael 569*","Oxford, Bodleian Library",NaN,False,0.0,243.0,150.0,original,0,1375.0,1400.0,0.0,200.0,current,43.0,parchment,fragment,NaN,NaN,2.0
762,"Madrid, Biblioteca Nacional, 22644","Madrid, Biblioteca Nacional",NaN,True,0.0,215.0,155.0,original,26,1400.0,1410.0,0.0,0.0,current,24.0,paper,fragment,Spain,Castilla,2.0
381,"Brussels, Koninklijke Bibliotheek, 9086-9087 [...","Brussels, Koninklijke Bibliotheek",NaN,False,0.0,NaN,NaN,NaN,0,1475.0,1500.0,398.0,0.0,current,NaN,paper,codex,NaN,NaN,NaN
430,"Krakau, Biblioteka Jagiellonska, Gall. Fol. 217","Krakau, Biblioteka Jagiellonska",textualis,False,0.0,NaN,NaN,NaN,0,1200.0,1300.0,0.0,275.0,current,NaN,parchment,fragment,NaN,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,"Paris, Bibliothèque Nationale de France, fr. 1...","Paris, Bibliothèque Nationale de France",cursive,False,182086.0,NaN,NaN,NaN,0,1460.0,1480.0,503.0,362.0,current,50.0,parchment,codex,France,Central France,2.0
1043,"Heidelberg, Universitätsbibliothek, Cpg 147","Heidelberg, Universitätsbibliothek",NaN,False,192618.0,NaN,NaN,NaN,0,1460.0,1480.0,522.0,369.0,current,NaN,parchment,codex,NaN,NaN,NaN
575,"Munich, Staatsbibliothek, cgm.1","Munich, Staatsbibliothek",NaN,False,194400.0,NaN,NaN,NaN,0,1475.0,1500.0,540.0,360.0,current,NaN,parchment,codex,NaN,NaN,NaN
938,"Linz, Landesarchiv, Pa I/3 b","Linz, Landesarchiv",NaN,False,204375.0,407.0,249.0,original,0,1350.0,1400.0,545.0,375.0,current,38.0,parchment,fragment,NaN,NaN,2.0


... but notice that some of `surface` entries are 0, because one of `leaf-height` or `leaf-width` was mistakenly entered as 0 instead of NA. Let's clean that up.

The main tool for subsetting by conditions inherently uses a boolean Series under the hood, but you can think of it like this: "return the rows of `df_valid` at the indices where `df_valid.surface` (a column) is greater than zero"

In [22]:
df_valid = df_valid[df_valid.surface > 0]
df_valid

,signature,repository,script,illustrated,surface,text-height,text-width,text-src,illustrations,date_min,date_max,leaf-height,leaf-width,leaf-src,number-lines,material,physical-type,country,region,columns
1,"Schloss Anholt (Isselburg), Fürstl. Salm-Salm'...","Schloss Anholt (Isselburg), Fürstl. Salm-Salm'...",textualis,False,66220.0,220.0,164.0,original,0,1320.0,1325.0,308.0,215.0,current,32.0,parchment,fragment,NaN,NaN,2.0
3,"Linz, Landesarchiv, 96/IV/54","Linz, Landesarchiv",NaN,False,56000.0,212.0,163.0,original,0,1390.0,1410.0,280.0,200.0,current,31.0,paper,codex,NaN,NaN,2.0
4,"Brussels, Koninklijke Bibliotheek, II 115,2","Brussels, Koninklijke Bibliotheek",textualis,False,35650.0,185.0,132.0,original,0,1340.0,1360.0,230.0,155.0,original,39.0,parchment,fragment,Southern Netherlands,Flanders,2.0
5,"Munich, Staatsbibliothek, cgm. 5249 / 3b","Munich, Staatsbibliothek",textualis,False,37120.0,195.0,127.0,original,0,1275.0,1300.0,232.0,160.0,current,42.0,parchment,fragment,NaN,NaN,2.0
9,"Escorial, Real Biblioteca del Monasterio, M. I...","Escorial, Real Biblioteca del Monasterio",textualis,False,33280.0,224.0,128.0,original,0,1275.0,1300.0,260.0,128.0,current,45.0,parchment,codex,France,Normandy,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160,"Marburg, Staatsarchiv, Best. 147 Hr 1, 16","Marburg, Staatsarchiv",NaN,False,15200.0,250.0,190.0,original,0,1200.0,1300.0,95.0,160.0,original,43.0,parchment,fragment,NaN,NaN,2.0
1162,"Orléans, Bibliothèque Municipale, -","Orléans, Bibliothèque Municipale",NaN,False,15750.0,NaN,NaN,NaN,0,1200.0,1300.0,150.0,105.0,current,30.0,parchment,fragment,NaN,NaN,1.0
1163,"Freiburg, Universitätsbibliothek, 513","Freiburg, Universitätsbibliothek",NaN,False,22400.0,150.0,115.0,original,0,1290.0,1310.0,160.0,140.0,current,NaN,parchment,fragment,NaN,NaN,2.0
1164,"Paris, Bibliothèque Nationale de France, fr. 340","Paris, Bibliothèque Nationale de France",NaN,True,126000.0,NaN,NaN,NaN,76,1400.0,1425.0,420.0,300.0,current,53.0,parchment,codex,NaN,NaN,3.0


## Grouping and aggregating

A very powerful method is pandas `groupby()` which creates a special grouping object that can be used for a lot of things. For example, let's see how many illustrated manuscripts occur for each script type...

In [23]:
df_valid.groupby("script")["illustrated"].value_counts()

script               illustrated
cursive              False           77
                     True            25
cursive - hybrida    False            1
cursive - textualis  True             3
hybrida              True            12
                     False            3
hybrida - textualis  True             1
other                False            1
textualis            False          193
                     True           102
Name: count, dtype: int64

We can also "normalize" these numbers by count to obtain comparable proportions

In [24]:
df_valid.groupby("script")["illustrated"].value_counts(normalize=True)

script               illustrated
cursive              False          0.754902
                     True           0.245098
cursive - hybrida    False          1.000000
cursive - textualis  True           1.000000
hybrida              True           0.800000
                     False          0.200000
hybrida - textualis  True           1.000000
other                False          1.000000
textualis            False          0.654237
                     True           0.345763
Name: proportion, dtype: float64

Finally, and this is just scratching the surface, we can "aggregate" columns from the groups using any function. Common functions (mean, median, std...) can be specific just by their name...

In [25]:
df_valid.groupby("script")["surface"].agg("mean")

script
cursive                 71919.284314
cursive - hybrida       47500.000000
cursive - textualis     54940.000000
hybrida                112492.733333
hybrida - textualis     58212.000000
other                   25024.000000
textualis               66349.711864
Name: surface, dtype: float64

We can subset our valid dataframe again by directly using the boolean column `illustrated` that we created earlier. Now we're down to just 186 manuscripts where we have illustrations as well as valid page sizes.

In [26]:
df_illustrated = df_valid[df_valid.illustrated]
df_illustrated

,signature,repository,script,illustrated,surface,text-height,text-width,text-src,illustrations,date_min,date_max,leaf-height,leaf-width,leaf-src,number-lines,material,physical-type,country,region,columns
16,"Modena, Biblioteca Estenze, E 59 (alfa T.3.11)","Modena, Biblioteca Estenze",textualis,True,150500.0,NaN,NaN,NaN,27,1300.0,1325.0,430.0,350.0,current,NaN,parchment,codex,Italy,Naples,2.0
23,"Paris, Bibliothèque Nationale de France, nouv....","Paris, Bibliothèque Nationale de France",cursive,True,66976.0,NaN,NaN,NaN,1,1350.0,1375.0,299.0,224.0,current,49.0,parchment,fragment,NaN,NaN,2.0
24,"New York, Pierpont Morgan Library, 41","New York, Pierpont Morgan Library",hybrida,True,142352.0,330.0,235.0,original,80,1468.0,1468.0,434.0,328.0,current,53.0,parchment,codex,France,Bourges,2.0
27,"London, British Library, Add. 5474","London, British Library",textualis,True,90068.0,NaN,NaN,NaN,26,1280.0,1300.0,356.0,253.0,current,48.0,parchment,codex,France,Therouanne,2.0
34,"London, British Library, Royal 20 A II","London, British Library",cursive - textualis,True,35340.0,168.0,95.0,original,19,1307.0,1330.0,228.0,155.0,current,30.0,parchment,codex,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,"London, British Library, Royal 19 E III","London, British Library",NaN,True,166250.0,300.0,225.0,original,7,1475.0,1500.0,475.0,350.0,current,37.0,parchment,codex,Southern Netherlands,Bruges,2.0
1140,"Paris, Bibliothèque Nationale de France, fr. 2...","Paris, Bibliothèque Nationale de France",NaN,True,69300.0,NaN,NaN,NaN,71,1280.0,1290.0,315.0,220.0,current,40.0,NaN,codex,Southern Netherlands,Thérouanne,2.0
1147,"Paris, Bibliothèque Nationale de France, fr. 794","Paris, Bibliothèque Nationale de France",textualis,True,76160.0,265.0,193.0,original,1,1214.0,1235.0,320.0,238.0,current,35.0,parchment,codex,France,Champagne,3.0
1158,"Paris, Bibliothèque Nationale de France, fr. 758","Paris, Bibliothèque Nationale de France",NaN,True,73600.0,NaN,NaN,NaN,1,1200.0,1300.0,320.0,230.0,current,40.0,parchment,codex,Southern Netherlands,Arras,2.0


### Other formats

Finally, we use pandas again to read the same data from Microsoft Excel.

In [27]:
try:
    import openpyxl
except ImportError:
    %pip install openpyxl

In [28]:
dfxl = pd.read_excel("../../datasets/arthur/manuscripts.xlsx")
dfxl

,signature,repository,script,text-height,text-width,text-src,illustrations,date_min,date_max,leaf-height,leaf-width,leaf-src,number-lines,material,physical-type,country,region,columns,narratives
0,"Schloss Anholt (Isselburg), Fürstl. Salm-Salm'...","Schloss Anholt (Isselburg), Fürstl. Salm-Salm'...",textualis,220.0,164.0,original,0,1320.0,1325.0,308.0,215.0,current,32.0,parchment,fragment,NaN,NaN,2.0,NAR0000000003
1,"Berlin, Staatsbibliothek, germ. fol. 923, 38","Berlin, Staatsbibliothek",NaN,NaN,NaN,NaN,0,1300.0,1400.0,NaN,NaN,NaN,NaN,parchment,fragment,Germany,Bayern,NaN,NAR0000000003
2,"Linz, Landesarchiv, 96/IV/54","Linz, Landesarchiv",NaN,212.0,163.0,original,0,1390.0,1410.0,280.0,200.0,current,31.0,paper,codex,NaN,NaN,2.0,NAR0000000127
3,"Brussels, Koninklijke Bibliotheek, II 115,2","Brussels, Koninklijke Bibliotheek",textualis,185.0,132.0,original,0,1340.0,1360.0,230.0,155.0,original,39.0,parchment,fragment,Southern Netherlands,Flanders,2.0,NAR0000000236
4,"Munich, Staatsbibliothek, cgm. 5249 / 3b","Munich, Staatsbibliothek",textualis,195.0,127.0,original,0,1275.0,1300.0,232.0,160.0,current,42.0,parchment,fragment,NaN,NaN,2.0,NAR0000000003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1161,"Orléans, Bibliothèque Municipale, -","Orléans, Bibliothèque Municipale",NaN,NaN,NaN,NaN,0,1200.0,1300.0,150.0,105.0,current,30.0,parchment,fragment,NaN,NaN,1.0,NAR0000000324
1162,"Freiburg, Universitätsbibliothek, 513","Freiburg, Universitätsbibliothek",NaN,150.0,115.0,original,0,1290.0,1310.0,160.0,140.0,current,NaN,parchment,fragment,NaN,NaN,2.0,NAR0000000032
1163,"Paris, Bibliothèque Nationale de France, fr. 340","Paris, Bibliothèque Nationale de France",NaN,NaN,NaN,NaN,76,1400.0,1425.0,420.0,300.0,current,53.0,parchment,codex,NaN,NaN,3.0,NAR0000000225 -- NAR0000000260
1164,"Heidelberg, Universitätsbibliothek, Cpg 397","Heidelberg, Universitätsbibliothek",textualis,165.0,100.0,original,0,1225.0,1250.0,198.0,122.0,current,26.0,parchment,codex,Germany,Rheinfranken,1.0,NAR0000000004


Oh no! The Excel indices start from 0, but the CSV we used before started from 1 (maybe it was created in R or Matlab). We can just use our vector operation trick again to add one to the entire index column (ie to each row in it)

In [29]:
dfxl.index += 1

Now (with the indices matching), we use the index locations from our `df_illustrated` dataframe but we grab the rows straight from the new Excel dataframe!

In [30]:
dfxl.loc[df_illustrated.index]

,signature,repository,script,text-height,text-width,text-src,illustrations,date_min,date_max,leaf-height,leaf-width,leaf-src,number-lines,material,physical-type,country,region,columns,narratives
16,"Modena, Biblioteca Estenze, E 59 (alfa T.3.11)","Modena, Biblioteca Estenze",textualis,NaN,NaN,NaN,27,1300.0,1325.0,430.0,350.0,current,NaN,parchment,codex,Italy,Naples,2.0,NAR0000000028
23,"Paris, Bibliothèque Nationale de France, nouv....","Paris, Bibliothèque Nationale de France",cursive,NaN,NaN,NaN,1,1350.0,1375.0,299.0,224.0,current,49.0,parchment,fragment,NaN,NaN,2.0,NAR0000000322 -- NAR0000000326 -- NAR0000000205
24,"New York, Pierpont Morgan Library, 41","New York, Pierpont Morgan Library",hybrida,330.0,235.0,original,80,1468.0,1468.0,434.0,328.0,current,53.0,parchment,codex,France,Bourges,2.0,NAR0000000263 -- NAR0000000323 -- NAR0000000028
27,"London, British Library, Add. 5474","London, British Library",textualis,NaN,NaN,NaN,26,1280.0,1300.0,356.0,253.0,current,48.0,parchment,codex,France,Therouanne,2.0,NAR0000000263 -- NAR0000000322 -- NAR0000000028
34,"London, British Library, Royal 20 A II","London, British Library",cursive - textualis,168.0,95.0,original,19,1307.0,1330.0,228.0,155.0,current,30.0,parchment,codex,NaN,NaN,2.0,NAR0000000323 -- NAR0000000322 -- NAR0000000273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,"London, British Library, Royal 19 E III","London, British Library",NaN,300.0,225.0,original,7,1475.0,1500.0,475.0,350.0,current,37.0,parchment,codex,Southern Netherlands,Bruges,2.0,NAR0000000240
1140,"Paris, Bibliothèque Nationale de France, fr. 2...","Paris, Bibliothèque Nationale de France",NaN,NaN,NaN,NaN,71,1280.0,1290.0,315.0,220.0,current,40.0,NaN,codex,Southern Netherlands,Thérouanne,2.0,NAR0000000326 -- NAR0000000325 -- NAR0000000324
1147,"Paris, Bibliothèque Nationale de France, fr. 794","Paris, Bibliothèque Nationale de France",textualis,265.0,193.0,original,1,1214.0,1235.0,320.0,238.0,current,35.0,parchment,codex,France,Champagne,3.0,NAR0000000046 -- NAR0000000105 -- NAR000000019...
1158,"Paris, Bibliothèque Nationale de France, fr. 758","Paris, Bibliothèque Nationale de France",NaN,NaN,NaN,NaN,1,1200.0,1300.0,320.0,230.0,current,40.0,parchment,codex,Southern Netherlands,Arras,2.0,NAR0000000323 -- NAR0000000327 -- NAR0000000028


And finally, a little taste of validation and cleaning. We can use `.equals()` to compare entire `Series` or `DataFrame` objects. Here we find out that the `signature` columns are *not* equal, even though they should be

In [31]:
dfxl.loc[df_illustrated.index]["signature"].equals(df_illustrated["signature"])

False

A different method, `.eq()`, returns a boolean `Series` instead (with an equality value for every row). By using a sneaky trick, we assign that to a variable, and then invert it using logical NOT (the tilde `~` operator in pandas). In other words, the original series was `True` almost everywhere and `False` in a couple of places, we swap that, so it is `True` just for the problem rows, and see what indices they are.

Take your time with this idea, it is not simple to understand at first. It can help to make new cells, and then look at individual variables. What is `s`? What does `~s` look like? etc.

In [32]:
s = dfxl.loc[df_illustrated.index]["signature"].eq(df_illustrated["signature"])
df_illustrated[~s]

,signature,repository,script,illustrated,surface,text-height,text-width,text-src,illustrations,date_min,date_max,leaf-height,leaf-width,leaf-src,number-lines,material,physical-type,country,region,columns
308,"Hamburg, Antiquariat Günther,","Hamburg, Antiquariat Günther",NaN,True,9702.0,NaN,NaN,NaN,1,1434.0,1454.0,98.0,99.0,current,NaN,parchment,fragment,France,Paris,NaN
596,"Bologna, Archivio della Fabbriceria della chie...","Bologna, Archivio della Fabbriceria della chie...",textualis,True,75905.0,255.0,160.0,original,1,1290.0,1310.0,323.0,235.0,current,40.0,parchment,fragment,NaN,NaN,2.0


... and let's take a look. Oh, this old trick -- someone has accidentally left an extra space at the end of two column entries. This kind of problem comes up all the time in data analysis, and requires constant vigilance. Fixing these problems is called "data cleaning" and,unfortunately, is a necessary process for almost all real-world work.

In [33]:
dfxl.loc[308]["signature"]

'Hamburg, Antiquariat Günther, '

In [34]:
df_illustrated.loc[308]["signature"]

'Hamburg, Antiquariat Günther,'

And to finish, a little trick with Python "f-strings". It is not as complicated as it looks, see if you can figure it out!

(Again, try breaking it into smaller pieces, and then putting them back together!)

In [35]:
print(
    f"A: {repr(dfxl.loc[596]['signature'])}\nB: {repr(df_illustrated.loc[596]['signature'])}"
)

A: 'Bologna, Archivio della Fabbriceria della chiesa di S. Petronio, '
B: 'Bologna, Archivio della Fabbriceria della chiesa di S. Petronio,'


```
Version History

Current: v1.0.3

10/9/24: 1.0.0: first draft, BN
18/9/24: 1.0.1: add group_by examples, BN
04/10/24: 1.0.2: correct typos and proofread, MK
13/10/24: 1.0.3: move to public, BN
```